# Differential expression analysis (DEA)

In [ ]:
library_load <- suppressMessages(
    
    list(
        
        # Seurat 
        library(Seurat), 
        library(SeuratWrappers), 
        
        # TradeSeq 
        library(tradeSeq), 
        library(clusterExperiment), 
        
        # Biomart 
        library(biomaRt), 
        
        # GSEA 
        library(msigdbr), 
        library(fgsea), 
        
        # Data 
        library(tidyverse), 
        library(openxlsx), 
        
        # Plotting 
        library(ComplexHeatmap), 
        library(circlize), 
        library(viridis), 
        library(ggplotify), 
        library(ComplexUpset), 
        
        # Python 
        library(reticulate)
        
    )
)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
options(warn=-1)

In [ ]:
ht_opt$message=FALSE

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")
source("bin/seurat_qc.R")
source("bin/seurat_dea.R")
source("bin/cell_type.R")
source("bin/gsea.R")

In [ ]:
# Plotting Theme
ggplot2::theme_set(theme_global_set()) # From project global source()

# Parameter settings

In [ ]:
so_file <- "data/object/pp.rds"

# Import

In [ ]:
# Seurat object
so <- readRDS(so_file)

# DEA 
dea_treatment_cell_type_fine <- dea_seurat(so, ident="cell_type_fine",  map=cell_type, "result/dea/dea_treatment_cell_type_fine", only_pos=FALSE, logfc_threshold=0, min_pct=0, cnt_min=1, cell_min=3, conserved=FALSE, treatment=TRUE, grouping_var="treatment", test_use="wilcox", compute=FALSE)

# Interferon gamma

In [ ]:
dea_mac_gamma <- readRDS("data/inf_signature_mac/DEA_IFN_gamma_mgi_symbol.rds")
dea_mac_gamma <- split(dea_mac_gamma, f=dea_mac_gamma$group)
dea_mac_gamma <- dea_mac_gamma[paste0("IFN_gamma_", c(2, 4, 6, 8, 24), "h")]

In [ ]:
options(repr.plot.width=5*5, repr.plot.height=5)

vp_dea_mac_gamma <- lapply(dea_mac_gamma, function(dea) {vp_dea(dea, log2_thold=1.5, adjpvalue_thold=0.05, top_label=10, title=unique(dea$group), conserved=FALSE)})
wrap_plots(vp_dea_mac_gamma, ncol=5, nrow=1) 

In [ ]:
dea_inf_gamma <- lapply(dea_mac_gamma, function(dea) {dea[dea$avg_log2FC>=1.5 & dea$p_val_adj <= 0.05, ]})
saveRDS(dea_inf_gamma, "result/inf_signature/dea_inf_gamma_mac.rds")
inf_genes_gamma <- unique(do.call("rbind", dea_inf_gamma)$gene)

In [ ]:
data <- lapply(dea_inf_gamma, function(dea) {x = data.frame(x=inf_genes_gamma %in% dea$gene); colnames(x) <- unique(dea$group); return(x)})
data <- do.call(cbind, data)

In [ ]:
upset(data=data, intersect=rev(colnames(data)), width_ratio=0.1, sort_sets=FALSE)

In [ ]:
inf_genes_gamma_2h <- dea_inf_gamma[["IFN_gamma_2h"]]$gene
inf_genes_gamma_4h <- dea_inf_gamma[["IFN_gamma_4h"]]$gene
inf_genes_gamma_6h <- dea_inf_gamma[["IFN_gamma_6h"]]$gene
inf_genes_gamma_8h <- dea_inf_gamma[["IFN_gamma_8h"]]$gene
inf_genes_gamma_24h <- dea_inf_gamma[["IFN_gamma_24h"]]$gene

In [ ]:
so_nacl <- subset(so, subset=treatment=="NaCl")
so_nacl <- AddModuleScore(so_nacl, features=list(inf_genes_gamma_2h[inf_genes_gamma_2h %in% rownames(so)]), assay="RNA", name="msINF_gamma_2h")
so_nacl <- AddModuleScore(so_nacl, features=list(inf_genes_gamma_4h[inf_genes_gamma_4h %in% rownames(so)]), assay="RNA", name="msINF_gamma_4h")
so_nacl <- AddModuleScore(so_nacl, features=list(inf_genes_gamma_6h[inf_genes_gamma_6h %in% rownames(so)]), assay="RNA", name="msINF_gamma_6h")
so_nacl <- AddModuleScore(so_nacl, features=list(inf_genes_gamma_8h[inf_genes_gamma_8h %in% rownames(so)]), assay="RNA", name="msINF_gamma_8h")
so_nacl <- AddModuleScore(so_nacl, features=list(inf_genes_gamma_24h[inf_genes_gamma_24h %in% rownames(so)]), assay="RNA", name="msINF_gamma_24h")
    
so_cpg <- subset(so, subset=treatment=="CpG")
so_cpg <- AddModuleScore(so_cpg, features=list(inf_genes_gamma_2h[inf_genes_gamma_2h %in% rownames(so)]), assay="RNA", name="msINF_gamma_2h")
so_cpg <- AddModuleScore(so_cpg, features=list(inf_genes_gamma_4h[inf_genes_gamma_4h %in% rownames(so)]), assay="RNA", name="msINF_gamma_4h")
so_cpg <- AddModuleScore(so_cpg, features=list(inf_genes_gamma_6h[inf_genes_gamma_6h %in% rownames(so)]), assay="RNA", name="msINF_gamma_6h")
so_cpg <- AddModuleScore(so_cpg, features=list(inf_genes_gamma_8h[inf_genes_gamma_8h %in% rownames(so)]), assay="RNA", name="msINF_gamma_8h")
so_cpg <- AddModuleScore(so_cpg, features=list(inf_genes_gamma_24h[inf_genes_gamma_24h %in% rownames(so)]), assay="RNA", name="msINF_gamma_24h")

ms_inf_gamma_nacl <- so_nacl@meta.data[, c("msINF_gamma_2h1", "msINF_gamma_4h1", "msINF_gamma_6h1", "msINF_gamma_8h1", "msINF_gamma_24h1")]
ms_inf_gamma_cpg <- so_cpg@meta.data[, c("msINF_gamma_2h1", "msINF_gamma_4h1", "msINF_gamma_6h1", "msINF_gamma_8h1", "msINF_gamma_24h1")]
ms_inf_gamma <- rbind(ms_inf_gamma_nacl, ms_inf_gamma_cpg)
colnames(ms_inf_gamma) <- gsub("h1", "h", colnames(ms_inf_gamma))

so <- AddMetaData(so, ms_inf_gamma)

## Module score UMAP

In [ ]:
fplot_1 <- suppressMessages(fplot(subset(so, subset=treatment=="NaCl"), reduction="umap", feature="msINF_gamma_2h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_2 <- suppressMessages(fplot(subset(so, subset=treatment=="NaCl"), reduction="umap", feature="msINF_gamma_4h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_3 <- suppressMessages(fplot(subset(so, subset=treatment=="NaCl"), reduction="umap", feature="msINF_gamma_6h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_4 <- suppressMessages(fplot(subset(so, subset=treatment=="NaCl"), reduction="umap", feature="msINF_gamma_8h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_5 <- suppressMessages(fplot(subset(so, subset=treatment=="NaCl"), reduction="umap", feature="msINF_gamma_24h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))

In [ ]:
options(repr.plot.width=5*5, repr.plot.height=5)

fplot_1 + fplot_2 + fplot_3 + fplot_4 + fplot_5 + plot_layout(ncol=5)

In [ ]:
fplot_1 <- suppressMessages(fplot(subset(so, subset=treatment=="CpG"), reduction="umap", feature="msINF_gamma_2h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_2 <- suppressMessages(fplot(subset(so, subset=treatment=="CpG"), reduction="umap", feature="msINF_gamma_4h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_3 <- suppressMessages(fplot(subset(so, subset=treatment=="CpG"), reduction="umap", feature="msINF_gamma_6h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_4 <- suppressMessages(fplot(subset(so, subset=treatment=="CpG"), reduction="umap", feature="msINF_gamma_8h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_5 <- suppressMessages(fplot(subset(so, subset=treatment=="CpG"), reduction="umap", feature="msINF_gamma_24h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))

In [ ]:
options(repr.plot.width=5*5, repr.plot.height=5)

fplot_1 + fplot_2 + fplot_3 + fplot_4 + fplot_5 + plot_layout(ncol=5)

## Module score box plot

In [ ]:
options(repr.plot.width=6*5, repr.plot.height=2.5)

ggplot(so@meta.data, aes(x=treatment, y=msINF_gamma_2h, color=treatment)) + 
    geom_boxplot() + 
    facet_grid(~cell_type_fine) + 
    scale_color_manual(values=color$treatment)

In [ ]:
options(repr.plot.width=6*5, repr.plot.height=2.5)

ggplot(so@meta.data, aes(x=treatment, y=msINF_gamma_4h, color=treatment)) + 
    geom_boxplot() + 
    facet_grid(~cell_type_fine) + 
    scale_color_manual(values=color$treatment)

In [ ]:
options(repr.plot.width=6*5, repr.plot.height=2.5)

ggplot(so@meta.data, aes(x=treatment, y=msINF_gamma_6h, color=treatment)) + 
    geom_boxplot() + 
    facet_grid(~cell_type_fine) + 
    scale_color_manual(values=color$treatment)

In [ ]:
options(repr.plot.width=6*5, repr.plot.height=2.5)

ggplot(so@meta.data, aes(x=treatment, y=msINF_gamma_8h, color=treatment)) + 
    geom_boxplot() + 
    facet_grid(~cell_type_fine) + 
    scale_color_manual(values=color$treatment)

In [ ]:
options(repr.plot.width=6*5, repr.plot.height=2.5)

ggplot(so@meta.data, aes(x=treatment, y=msINF_gamma_24h, color=treatment)) + 
    geom_boxplot() + 
    facet_grid(~cell_type_fine) + 
    scale_color_manual(values=color$treatment)

## Gene set enrichment analysis

In [ ]:
inf_genes_gamma <- lapply(dea_inf_gamma, function(x) x$gene)

In [ ]:
options(repr.plot.width=10*2.5, repr.plot.height=2.5*ceiling(length(dea_treatment_cell_type_fine)/10))

gsea_treatment_cell_type_fine <- lapply(1:length(dea_treatment_cell_type_fine), function(i) gsea_from_dea(dea=dea_treatment_cell_type_fine[[i]], gene_sets=inf_genes_gamma, ident=names(dea_treatment_cell_type_fine)[i]))
plot_gsea_treatment_cell_type_fine <- lapply(gsea_treatment_cell_type_fine, function(x) {gsea_plot(x, padj_thr=0.1)})
wrap_plots(plot_gsea_treatment_cell_type_fine, ncol=10)

# Interferon beta 

In [ ]:
dea_mac_beta <- readRDS("data/inf_signature_mac/DEA_IFN_beta_mgi_symbol.rds")
dea_mac_beta <- split(dea_mac_beta, f=dea_mac_beta$group)
dea_mac_beta <- dea_mac_beta[paste0("IFN_beta_", c(2, 4, 6, 8, 24), "h")]

In [ ]:
options(repr.plot.width=5*5, repr.plot.height=5)

vp_dea_mac_beta <- lapply(dea_mac_beta, function(dea) {vp_dea(dea, log2_thold=1.5, adjpvalue_thold=0.05, top_label=10, title=unique(dea$group), conserved=FALSE)})
wrap_plots(vp_dea_mac_beta, ncol=5, nrow=1) 

In [ ]:
dea_inf_beta <- lapply(dea_mac_beta, function(dea) {dea[dea$avg_log2FC>=1.5 & dea$p_val_adj <= 0.05, ]})
saveRDS(dea_inf_beta, "result/inf_signature/dea_inf_beta_mac.rds")
inf_genes_beta <- unique(do.call("rbind", dea_inf_beta)$gene)

In [ ]:
data <- lapply(dea_inf_beta, function(dea) {x = data.frame(x=inf_genes_beta %in% dea$gene); colnames(x) <- unique(dea$group); return(x)})
data <- do.call(cbind, data)

In [ ]:
upset(data=data, intersect=rev(colnames(data)), width_ratio=0.1, sort_sets=FALSE)

In [ ]:
inf_genes_beta_2h <- dea_inf_beta[["IFN_beta_2h"]]$gene
inf_genes_beta_4h <- dea_inf_beta[["IFN_beta_4h"]]$gene
inf_genes_beta_6h <- dea_inf_beta[["IFN_beta_6h"]]$gene
inf_genes_beta_8h <- dea_inf_beta[["IFN_beta_8h"]]$gene
inf_genes_beta_24h <- dea_inf_beta[["IFN_beta_24h"]]$gene

In [ ]:
so_nacl <- subset(so, subset=treatment=="NaCl")
so_nacl <- AddModuleScore(so_nacl, features=list(inf_genes_beta_2h[inf_genes_beta_2h %in% rownames(so)]), assay="RNA", name="msINF_beta_2h")
so_nacl <- AddModuleScore(so_nacl, features=list(inf_genes_beta_4h[inf_genes_beta_4h %in% rownames(so)]), assay="RNA", name="msINF_beta_4h")
so_nacl <- AddModuleScore(so_nacl, features=list(inf_genes_beta_6h[inf_genes_beta_6h %in% rownames(so)]), assay="RNA", name="msINF_beta_6h")
so_nacl <- AddModuleScore(so_nacl, features=list(inf_genes_beta_8h[inf_genes_beta_8h %in% rownames(so)]), assay="RNA", name="msINF_beta_8h")
so_nacl <- AddModuleScore(so_nacl, features=list(inf_genes_beta_24h[inf_genes_beta_24h %in% rownames(so)]), assay="RNA", name="msINF_beta_24h")
    
so_cpg <- subset(so, subset=treatment=="CpG")
so_cpg <- AddModuleScore(so_cpg, features=list(inf_genes_beta_2h[inf_genes_beta_2h %in% rownames(so)]), assay="RNA", name="msINF_beta_2h")
so_cpg <- AddModuleScore(so_cpg, features=list(inf_genes_beta_4h[inf_genes_beta_4h %in% rownames(so)]), assay="RNA", name="msINF_beta_4h")
so_cpg <- AddModuleScore(so_cpg, features=list(inf_genes_beta_6h[inf_genes_beta_6h %in% rownames(so)]), assay="RNA", name="msINF_beta_6h")
so_cpg <- AddModuleScore(so_cpg, features=list(inf_genes_beta_8h[inf_genes_beta_8h %in% rownames(so)]), assay="RNA", name="msINF_beta_8h")
so_cpg <- AddModuleScore(so_cpg, features=list(inf_genes_beta_24h[inf_genes_beta_24h %in% rownames(so)]), assay="RNA", name="msINF_beta_24h")

ms_inf_beta_nacl <- so_nacl@meta.data[, c("msINF_beta_2h1", "msINF_beta_4h1", "msINF_beta_6h1", "msINF_beta_8h1", "msINF_beta_24h1")]
ms_inf_beta_cpg <- so_cpg@meta.data[, c("msINF_beta_2h1", "msINF_beta_4h1", "msINF_beta_6h1", "msINF_beta_8h1", "msINF_beta_24h1")]
ms_inf_beta <- rbind(ms_inf_beta_nacl, ms_inf_beta_cpg)
colnames(ms_inf_beta) <- gsub("h1", "h", colnames(ms_inf_beta))

so <- AddMetaData(so, ms_inf_beta)

## Module score UMAP

In [ ]:
fplot_1 <- suppressMessages(fplot(subset(so, subset=treatment=="NaCl"), reduction="umap", feature="msINF_beta_2h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_2 <- suppressMessages(fplot(subset(so, subset=treatment=="NaCl"), reduction="umap", feature="msINF_beta_4h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_3 <- suppressMessages(fplot(subset(so, subset=treatment=="NaCl"), reduction="umap", feature="msINF_beta_6h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_4 <- suppressMessages(fplot(subset(so, subset=treatment=="NaCl"), reduction="umap", feature="msINF_beta_8h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_5 <- suppressMessages(fplot(subset(so, subset=treatment=="NaCl"), reduction="umap", feature="msINF_beta_24h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))

In [ ]:
options(repr.plot.width=5*5, repr.plot.height=5)

fplot_1 + fplot_2 + fplot_3 + fplot_4 + fplot_5 + plot_layout(ncol=5)

In [ ]:
fplot_1 <- suppressMessages(fplot(subset(so, subset=treatment=="CpG"), reduction="umap", feature="msINF_beta_2h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_2 <- suppressMessages(fplot(subset(so, subset=treatment=="CpG"), reduction="umap", feature="msINF_beta_4h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_3 <- suppressMessages(fplot(subset(so, subset=treatment=="CpG"), reduction="umap", feature="msINF_beta_6h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_4 <- suppressMessages(fplot(subset(so, subset=treatment=="CpG"), reduction="umap", feature="msINF_beta_8h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))
fplot_5 <- suppressMessages(fplot(subset(so, subset=treatment=="CpG"), reduction="umap", feature="msINF_beta_24h", alpha=1, pt_size=1) + scale_color_viridis(option="G"))

In [ ]:
options(repr.plot.width=5*5, repr.plot.height=5)

fplot_1 + fplot_2 + fplot_3 + fplot_4 + fplot_5 + plot_layout(ncol=5)

## Module score box plot

In [ ]:
so$treatment <- factor(so$treatment, levels=c("NaCl", "CpG"))
so$cell_type_fine <- factor(so$cell_type_fine, levels=cell_type_fine_order)

In [ ]:
options(repr.plot.width=6*5, repr.plot.height=2.5)

ggplot(so@meta.data, aes(x=treatment, y=msINF_beta_2h, color=treatment)) + 
    geom_boxplot() + 
    facet_grid(~cell_type_fine) + 
    scale_color_manual(values=color$treatment)

In [ ]:
options(repr.plot.width=6*5, repr.plot.height=2.5)

ggplot(so@meta.data, aes(x=treatment, y=msINF_beta_4h, color=treatment)) + 
    geom_boxplot() + 
    facet_grid(~cell_type_fine) + 
    scale_color_manual(values=color$treatment)

In [ ]:
options(repr.plot.width=6*5, repr.plot.height=2.5)

ggplot(so@meta.data, aes(x=treatment, y=msINF_beta_6h, color=treatment)) + 
    geom_boxplot() + 
    facet_grid(~cell_type_fine) + 
    scale_color_manual(values=color$treatment)

In [ ]:
options(repr.plot.width=6*5, repr.plot.height=2.5)

ggplot(so@meta.data, aes(x=treatment, y=msINF_beta_8h, color=treatment)) + 
    geom_boxplot() + 
    facet_grid(~cell_type_fine) + 
    scale_color_manual(values=color$treatment)

In [ ]:
options(repr.plot.width=6*5, repr.plot.height=2.5)

ggplot(so@meta.data, aes(x=treatment, y=msINF_beta_24h, color=treatment)) + 
    geom_boxplot() + 
    facet_grid(~cell_type_fine) + 
    scale_color_manual(values=color$treatment)

## Gene set enrichment analysis

In [ ]:
inf_genes_beta <- lapply(dea_inf_beta, function(x) x$gene)

In [ ]:
options(repr.plot.width=10*2.5, repr.plot.height=2.5*ceiling(length(dea_treatment_cell_type_fine)/10))

gsea_treatment_cell_type_fine <- lapply(1:length(dea_treatment_cell_type_fine), function(i) gsea_from_dea(dea=dea_treatment_cell_type_fine[[i]], gene_sets=inf_genes_beta, ident=names(dea_treatment_cell_type_fine)[i]))
plot_gsea_treatment_cell_type_fine <- lapply(gsea_treatment_cell_type_fine, function(x) {gsea_plot(x, padj_thr=0.1)})
wrap_plots(plot_gsea_treatment_cell_type_fine, ncol=10)

# DEA INF  

## Monocytes 

In [ ]:
so_mo <- subset(so, subset=cell_type_fine %in% c("cMo (1)", "cMo (2)", "RPMP"))
so_mo$group <- "Mo (CpG) vs Mo (NaCl)"

In [ ]:
dea_mo <- dea_seurat(so_mo, ident="group",  only_pos=FALSE, logfc_threshold=0, min_pct=0, cnt_min=1, cell_min=3, conserved=FALSE, treatment=TRUE, grouping_var="treatment", test_use="wilcox", gene_filter=unique(unlist(inf_genes_beta)), compute=TRUE)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

vp_mo <- lapply(seq_along(dea_mo), function(i) vp_dea(dea_mo[[i]], title=dea_mo[[i]]$ident, log2_thold=0.25, adjpvalue_thold=0.05))
wrap_plots(vp_mo, ncol=1, nrow=1) 

## RPM

In [ ]:
so_rpm <- subset(so, subset=cell_type_fine %in% c("RPM"))
so_rpm$group <- "RPM (CpG) vs RPM (NaCl)"

In [ ]:
dea_rpm <- dea_seurat(so_rpm, ident="group",  only_pos=FALSE, logfc_threshold=0, min_pct=0, cnt_min=1, cell_min=3, conserved=FALSE, treatment=TRUE, grouping_var="treatment", test_use="wilcox", gene_filter=unique(unlist(inf_genes_beta)), compute=TRUE)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

vp_rpm <- lapply(seq_along(dea_rpm), function(i) vp_dea(dea_rpm[[i]], title=dea_rpm[[i]]$ident, log2_thold=0.25, adjpvalue_thold=0.05))
wrap_plots(vp_rpm, ncol=1, nrow=1) 

## Myeloid

In [ ]:
so_m <- subset(so, subset=cell_type_fine %in% c("cMo (1)", "cMo (2)", "RPMP", "RPM"))
so_m$group <- "Myeloid (CpG) vs Myeloid (NaCl)"

In [ ]:
dea_m <- dea_seurat(so_m, ident="group",  only_pos=FALSE, logfc_threshold=0, min_pct=0, cnt_min=1, cell_min=3, conserved=FALSE, treatment=TRUE, grouping_var="treatment", test_use="wilcox", gene_filter=unique(unlist(inf_genes_beta)), compute=TRUE)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

vp_m <- lapply(seq_along(dea_m), function(i) vp_dea(dea_m[[i]], title=dea_m[[i]]$ident, log2_thold=0.25, adjpvalue_thold=0.05))
wrap_plots(vp_m, ncol=1, nrow=1) 